## Gemini API

In [1]:
# pip install google-genai

In [ ]:
import os
import time
from dotenv import load_dotenv
from google import genai

# === ENV SETUP ===
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=API_KEY)

# === CONFIGURATION ===
AUDIO_DIR = r"C:\...\Audio-Data"
OUTPUT_DIR = os.path.join(AUDIO_DIR, "transcripts")
MODEL = "models/gemini-2.5-flash"
DELAY_BETWEEN_REQUESTS = 6  # seconds (for Free Tier: 10 RPM)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# === TRANSCRIBE FUNCTION WITH RETRY & RATE LIMIT ===
def transcribe_audio(file_path):
    try:
        print(f"📤 Uploading {file_path}")
        uploaded = client.files.upload(file=file_path)

        print("⏳ Transcribing...")
        response = client.models.generate_content(
            model=MODEL,
            contents=[
                "Transcribe this call center audio with timestamps [00:00] and speakers 'Speaker:' and 'Operator:'.",
                uploaded
            ]
        )
        return response.text

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")
        return None

# === MAIN LOOP ===
for filename in os.listdir(AUDIO_DIR):
    if not filename.lower().endswith(".wav"):
        continue

    audio_path = os.path.join(AUDIO_DIR, filename)
    output_path = os.path.join(OUTPUT_DIR, os.path.splitext(filename)[0] + ".txt")

    # Skip if already transcribed
    if os.path.exists(output_path):
        print(f"✅ Already done: {filename}")
        continue

    transcription = transcribe_audio(audio_path)

    if transcription:
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(transcription)
        print(f"✅ Saved: {output_path}")
    else:
        print(f"⚠️ Failed to transcribe: {filename}")

    # Respect rate limits
    print(f"🕒 Sleeping {DELAY_BETWEEN_REQUESTS}s to respect rate limit...")
    time.sleep(DELAY_BETWEEN_REQUESTS)